In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime
import scipy.stats
from matplotlib.colors import ListedColormap
import math
import random
import requests
import datetime

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibrationDisplay
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.constraints import maxnorm
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, EarlyStopping
from tensorflow.keras.layers import Dense, Input, InputLayer, Add, Concatenate, Dropout, AlphaDropout
from tensorflow.keras.utils import plot_model
# from tensorflow_addons.activations import mish



In [2]:
constructor_standings = pd.read_csv("../input/formula-1-world-championship-1950-2020/constructor_standings.csv")
driver_standings = pd.read_csv("../input/formula-1-world-championship-1950-2020/driver_standings.csv")
pit_stops = pd.read_csv("../input/formula-1-world-championship-1950-2020/pit_stops.csv")


# Added
qualifying = pd.read_csv("../input/formula-1-world-championship-1950-2020/qualifying.csv")
races = pd.read_csv("../input/formula-1-world-championship-1950-2020/races.csv")
results = pd.read_csv("../input/formula-1-world-championship-1950-2020/results.csv")
lap_times = pd.read_csv("../input/formula-1-world-championship-1950-2020/lap_times.csv")

# Not Needed
seasons = pd.read_csv("../input/formula-1-world-championship-1950-2020/seasons.csv")
constructor_results = pd.read_csv("../input/formula-1-world-championship-1950-2020/constructor_results.csv") 
status = pd.read_csv("../input/formula-1-world-championship-1950-2020/status.csv")
drivers = pd.read_csv("../input/formula-1-world-championship-1950-2020/drivers.csv")
circuits = pd.read_csv("../input/formula-1-world-championship-1950-2020/circuits.csv")
constructors = pd.read_csv("../input/formula-1-world-championship-1950-2020/constructors.csv")

todo:

driv age
differences in quali time


In [3]:
print(results.shape)
rename_dict = {'fastestLap': 'fastestLapNumber', 'rank': 'fastestLapRank'}
results.rename(columns=rename_dict, inplace=True)
results.head(2)

(25460, 18)


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLapNumber,fastestLapRank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1


In [4]:
dropColumns = ['name', 'number', 'position', 'positionText', 'time_x', 'milliseconds', 'time_y', 'url', 'date', 'resultId']
raceResults = pd.merge(results, races, on='raceId').drop(dropColumns, axis=1)
raceResults.head(2)

,raceId,driverId,constructorId,grid,positionOrder,points,laps,fastestLapNumber,fastestLapRank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId
0,18,1,1,1,1,10.0,58,39,2,1:27.452,218.300,1,2008,1,1
1,18,2,2,5,2,8.0,58,41,3,1:27.739,217.586,1,2008,1,1


In [5]:
dropColumns = ['number', 'position', 'qualifyId']
qualifyingdrop = qualifying.drop(columns=dropColumns,axis=1)
qualifyingdrop.head(2)

,raceId,driverId,constructorId,q1,q2,q3
0,18,1,1,1:26.572,1:25.187,1:26.714
1,18,9,2,1:26.103,1:25.315,1:26.869


In [6]:
raceResultsQ = pd.merge(raceResults, qualifyingdrop, on=['raceId', 'driverId', 'constructorId'])
raceResultsQ.head(2)

,raceId,driverId,constructorId,grid,positionOrder,points,laps,fastestLapNumber,fastestLapRank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,q1,q2,q3
0,18,1,1,1,1,10.0,58,39,2,1:27.452,218.300,1,2008,1,1,1:26.572,1:25.187,1:26.714
1,18,2,2,5,2,8.0,58,41,3,1:27.739,217.586,1,2008,1,1,1:25.960,1:25.518,1:27.236


In [7]:
dropCol = ['position', 'time']
lap_times_piv = lap_times.drop(columns=dropCol, axis=1).pivot(index=['raceId', 'driverId'], columns='lap', values='milliseconds').reset_index()
lap_times_piv.head(2)

lap,raceId,driverId,1,2,3,4,5,6,7,8,...,78,79,80,81,82,83,84,85,86,87
0,1,1,109088.0,93740.0,91600.0,91067.0,92129.0,90469.0,89488.0,90302.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,157754.0,97640.0,91697.0,91867.0,92234.0,92036.0,91659.0,91801.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
raceResultsAll = pd.merge(raceResultsQ, lap_times_piv, on=['raceId', 'driverId'])
raceResultsAll.head(2)

,raceId,driverId,constructorId,grid,positionOrder,points,laps,fastestLapNumber,fastestLapRank,fastestLapTime,...,78,79,80,81,82,83,84,85,86,87
0,18,1,1,1,1,10.0,58,39,2,1:27.452,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18,2,2,5,2,8.0,58,41,3,1:27.739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
constructor_standings.head(2)

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
